The NPI is a unique identification number for covered health care providers.

In [1]:
import pandas as pd
npidata_s = pd.read_csv("/Users/cynding/Desktop/Brandeis 2022 Spring/HS 256F - Healthcare Data Analytics/HW1/NPPES_Data_Dissemination_January_2022/npidata_pfile_20050523-20220109.csv",usecols=['NPI','Provider License Number State Code_1','Provider Business Mailing Address State Name','Healthcare Provider Taxonomy Code_1','Provider Gender Code','Provider Last Name (Legal Name)','Provider First Name'])

## Question 1

MA

In [2]:
npidata_s[(npidata_s['Provider First Name']=='ROBERTA') & (npidata_s['Provider Last Name (Legal Name)']=='CAPLAN')]

,NPI,Provider Last Name (Legal Name),Provider First Name,Provider Business Mailing Address State Name,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Provider License Number State Code_1
1724076,1801936497,CAPLAN,ROBERTA,MA,F,103T00000X,MA


### Filter out the designated States

In [3]:
State_list = ['HI','MI','MN','MS','NY','OK','SD','TN']
npidata = npidata_s[npidata_s['Provider Business Mailing Address State Name'].isin(State_list)]
npidata.shape

(1202359, 7)

## Question 3

In [4]:
npidata['Healthcare Provider Taxonomy Code_1'].head()

7     231H00000X
11    207V00000X
12    363A00000X
17    208600000X
18    174400000X
Name: Healthcare Provider Taxonomy Code_1, dtype: object

### Low Risk
Obstetrics & Gynecology 207V00000X
Pediatrics 	208000000X

### High Risk
Surgery 	208600000X
Orthopaedic Surgery 	207X00000X

In [5]:
low_risk = ['207V00000X','208000000X']
high_risk = ['208600000X','207X00000X']

In [6]:
import scipy.stats

In [7]:
male_low_risk = len(npidata[(npidata['Provider Gender Code']=='M') & (npidata['Healthcare Provider Taxonomy Code_1'].isin(low_risk))])
male_high_risk = len(npidata[(npidata['Provider Gender Code']=='M') & (npidata['Healthcare Provider Taxonomy Code_1'].isin(high_risk))])

In [8]:
female_low_risk = len(npidata[(npidata['Provider Gender Code']=='F') & (npidata['Healthcare Provider Taxonomy Code_1'].isin(low_risk))])
female_high_risk = len(npidata[(npidata['Provider Gender Code']=='F') & (npidata['Healthcare Provider Taxonomy Code_1'].isin(high_risk))])

In [9]:
df = pd.DataFrame({'Risk':['High Risk','Low Risk'],
    'Male':[male_high_risk,male_low_risk],
    'Female':[female_high_risk,female_low_risk]
})
df.set_index('Risk')

,Male,Female
Risk,,
High Risk,8149,1626
Low Risk,7169,11903


Null = there is no association between practitioner gender and the taxonomy.

In [10]:
from scipy.stats import fisher_exact

In [11]:
oddsratio,pvalue=fisher_exact([[male_low_risk,male_high_risk], [female_low_risk,female_high_risk]])
pvalue

0.0

## Question 4

In [12]:
npidata_4 = pd.read_csv("/Users/cynding/Desktop/Brandeis 2022 Spring/HS 256F - Healthcare Data Analytics/HW1/NPPES_Data_Dissemination_January_2022/npidata_pfile_20050523-20220109.csv",usecols=['NPI','Entity Type Code','Healthcare Provider Taxonomy Code_1','Provider Business Mailing Address State Name'])

In [13]:
npidata_4 = npidata_4[(npidata_4['Entity Type Code']==2) & (npidata_4['Healthcare Provider Taxonomy Code_1']=='261QM1200X')]

In [14]:
mri = npidata_4.groupby(npidata_4['Provider Business Mailing Address State Name']).count()

In [15]:
mri = mri[['NPI']]
mri.rename_axis('State',inplace=True)

The population data by States was downloaded from the United States Census Bureau.

In [16]:
population = pd.read_excel('/Users/cynding/Desktop/Brandeis 2022 Spring/HS 256F - Healthcare Data Analytics/HW1/Population.xlsx')

In [17]:
population.head()

,Geographical Area,July 1 2020,State
0,Alabama,4921532,AL
1,Alaska,731158,AK
2,Arizona,7421401,AZ
3,Arkansas,3030522,AR
4,California,39368078,CA


In [18]:
population['population']=population['July 1 2020']/1000000
#population.set_index(['State'])

In [19]:
df = mri.merge(population.drop(columns=['Geographical Area','July 1 2020']),on='State')
df['NPI Density']=df['NPI']/df['population']

In [20]:
df.to_csv('NPI Density.csv')